### Initialize

In [13]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0]))
from tools import print_response, StopExecution, get_base_url, get_root_path
import esh_objects as esh

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]

def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}Example/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
            df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


### Search for a Person where the Location of its is in London
--> It is searched for a person where a location of a person is related to London

In [14]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relLocation', 'location', 'city']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='London'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relLocation.location.city:London'))
Total count: 1


,First Name,Last Name,Age
0,Joseph,Kennedy,36


## Search for a Person where the case is about money laundering
--> It is searched for a person who is involved in money laundering 

In [15]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relCases', 'cases', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy*')) #String search with spaces and semi colons not possible
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relCases.cases.entityDesc:Kennedy*'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Search for Persons where the case is about money laundering and the leads includes "payments"
--> This is an enhancment of the query before. It is searched for the payments of the money laundering

In [16]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relCases', 'cases', 'relLeads', 'lead', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy*')) #String search with spaces and semi colons not possible
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relCases.cases.relLeads.lead.entityDesc:Kennedy*'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Search for the time range of the payments when the person did the money laundering
--> This is an enhancement of the query before. It is searched for the time ranges when the payments were done

In [17]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relLeads', 'lead', 'entityTime']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[2013-01-01 2019-12-12]'))  # String search with spaces and semi colons not possible
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relCases.cases.relLeads.lead.entityTime:[2013-01-01 2019-12-12]'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Search for a case where the actvity was a phone intercept between J Kennedy and B Ramage

In [18]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Cases'),
            esh.Comparison(
                property=esh.Property(
                    property=['relActivity', 'activity', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Phone*intercept*'))  # String search with spaces and semi colons not possible
        ]))

print(f'ESH query statement: {so.to_statement()}')
# execute_search_query(so.dict(exclude_none=True))

r = requests.post(
    f'{base_url}/v1/query/{selected}Example/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'caseStartDate', 'caseSummary', 'priority']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time', 'caseStartDate': 'Start Date', 'caseSummary': 'Summary', 'priority': 'Priority'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Cases AND relActivity.activity.entityDesc:Phone*intercept*'))
Total count: 1


,Descritpion,Time,Start Date,Summary,Priority
0,Kennedy J: Obtaining a financial advantage by ...,2018-03-16 09:00:00,2018-03-16,Investigation into Joseph Kennedy for obtainin...,2


## Search for the objects SIM of a activity

In [19]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Activity'),
            esh.Comparison(
                property=esh.Property(
                    property=['relObject', 'object', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='*SIM*'))  # String search with spaces and semi colons not possible
        ]))

print(f'ESH query statement: {so.to_statement()}')
# execute_search_query(so.dict(exclude_none=True))

r = requests.post(
    f'{base_url}/v1/query/{selected}Example/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Activity AND relObject.object.entityDesc:*SIM*'))
Total count: 1


,Descritpion,Time
0,Phone intercept: J Kennedy - B Ramage,2018-04-24 13:00:00


## Search for a person who has an incident with "Threat of violence" at the location "Dingley Apartments, Braddon"

In [20]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'incident', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Threat*violence*')),  # String search with spaces and semi colons not possible
            esh.Comparison(
                property=esh.Property(property=['relIncident', 'incident', 'location']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Dingley*Apartments*')
            )    
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relIncident.incident.entityDesc:Threat*violence* AND relIncident.incident.location:Dingley*Apartments*'))
Total count: 1


,First Name,Last Name,Age
0,Rogerio,Gomez,42
